<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📏 תהליך הסבת שכבת חזיתות לבנק"ל מודרני 📏 

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
print('13. Parcels2DFronts:')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import re
import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:125%">
נתוני חזיתות ברצף

In [ ]:
arc_cols = ['RADIUS', 'FNODE_', 'TNODE_', 'LENGTH', 'LPOLY_', 'RPOLY_', 'geometry']

arc = daskgpd.read_file(path = CNFG.Arcstorm + 'PARCEL_ARC.gdb', layer = 'PARCEL_ARC', columns = arc_cols, npartitions = 8).compute()

del arc_cols

print('         input 1/6 read')

<h1 style="text-align:right;font-size:125%">
נתוני זיהוי של נקודות גבול

In [ ]:
BorderPoints_cols = ['PARCEL_SDE', 'GlobalID']

BorderPoints = daskgpd.read_file(CNFG.ProductPath, layer = 'BorderPoints', columns = BorderPoints_cols, npartitions = 8).compute()

del BorderPoints_cols

print('         input 2/6 read')

<h1 style="text-align:right;font-size:125%">
נתוני חזיתות של חלקות שומא ברצף

In [ ]:
TaxFronts = daskgpd.read_file(path = CNFG.CADSDE + 'PARCEL_ALL_SHUMA.gdb', layer = 'PARCEL_ALL_SHUMA_remains_Fronts', columns = ['geometry'], npartitions = 8).compute()

print('         input 3/6 read')

<h1 style="text-align:right;font-size:125%">
נתוני תהליכים יוצרים של חלקות

In [ ]:
Parcels_cols = ['PARCEL_SDE_', 'TALAR_NUMBER', 'TALAR_YEAR']

Parcels = daskgpd.read_file(path = CNFG.Arcstorm + 'PARCEL_POLYGON_SJOIN.gdb', layer = 'PARCEL_POLYGON_SJOIN', columns = Parcels_cols, npartitions = 8).compute()

del Parcels_cols

print('         input 4/6 read')

<h1 style="text-align:right;font-size:125%">
נתוני רישומי קדסטר

In [ ]:
Records_cols = ['ProcessName', 'ProcessType', 'DataSource', 'RecordGUID', 'ProcessGUID']

Records = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = Records_cols)

del Records_cols

print('         input 5/6 read')

<h1 style="text-align:right;font-size:125%">
נתוני תאריכי סטאטוס של תהליכי קדסטר

In [ ]:
Dates_cols = ['CPBUniqueID', 'DateStatus']

Dates = daskgpd.read_file(path = CNFG.ProductPath, layer = 'CPBStatusAndDates', columns = Dates_cols, npartitions = 8).compute()

del Dates_cols

print('         input 6/6 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️ 

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right; font-size:125%">
:עיבוד נתוני רישומי קדסטר
<h1 style="text-align:right; font-size:100%">
סינון לרישומים יוצרים של תצ"ר ופסק דין בלבד

In [ ]:
Records = Records[Records['RecordGUID'].notna()]

Records = Records[Records['ProcessType'].isin([1, 3])]

Records['ProcessYear'] = Records['ProcessName'].apply(lambda x: re.split('/|-', x)[1])

Records = Records.merge(Dates, left_on = 'ProcessGUID', right_on = 'CPBUniqueID', how = 'left')\
                 .drop(columns = ['CPBUniqueID', 'ProcessGUID', 'ProcessType'])\
                 .sort_values(['ProcessName', 'RecordGUID', 'DateStatus'])

Records = Records.groupby(by = 'ProcessName', as_index = False, dropna = False).max()

Records.rename(columns = {'RecordGUID' : 'CreatedByRecord'}, inplace = True)

del Dates

<h1 style="text-align:right; font-size:125%">
עיבוד נתוני חלקות
<h1 style="text-align:right; font-size:100%">
יצירת שדה של שם מפה מלא
מיזוג נתוני החלקות עם רישומי קדסטר לקבלת מזהה תהליך יוצר

In [ ]:
Parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = Parcels[['TALAR_NUMBER', 'TALAR_YEAR']].fillna(0)
Parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = Parcels[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)
Parcels['ProcessName'] = Parcels['TALAR_NUMBER'].astype(str) + '/' + Parcels['TALAR_YEAR'].astype(str)

Parcels = Parcels.merge(right = Records, on = 'ProcessName', how = 'left')
Parcels = Parcels[~Parcels['CreatedByRecord'].isna()]

Parcels.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR', 'ProcessName'], inplace = True)

del Records

<h1 style="text-align:right; font-size:125%">
:עיבוד נתוני חזיתות
<h1 style="text-align:right; font-size:100%">
הסרת חזיתות שגויות בעלות אורך מחושב אפסי   

In [ ]:
arc = arc[arc['LENGTH'] != 0.000000]

<h1 style="text-align:right; font-size:100%">    
תיקון ערכים שגויים בשדה רדיוס
ומילוי ערכים חסרים ברדיוס עם 0

In [ ]:
arc['RADIUS'].replace(['', ' ',' 0.000 0', ' 0.000 1', ' 0.000 2', ' 0.000 3'], 0.00, inplace = True)
arc['RADIUS'].fillna(0.0, inplace = True)

<h1 style="text-align:right; font-size:100%">    
תיקון רשת קואורדינטות לרשת ישראל התקפה

In [ ]:
arc.to_crs(2039, inplace = True)

<h1 style="text-align:right; font-size:100%">
תיקון סוגי שדות

In [ ]:
arc = arc.astype({'RADIUS' : float,
                  'FNODE_' : int,
                  'TNODE_' : int,
                  'LENGTH' : float,
                  'LPOLY_' : 'Int64',
                  'RPOLY_' : 'Int64'})

<h1 style="text-align:right; font-size:100%">
צירוף חזיתות שומא לחזיתות רצף

In [ ]:
arc = pd.concat([arc, TaxFronts], ignore_index = True).reset_index(drop = True)
del TaxFronts

<h1 style="text-align:right; font-size:100%">
הגדרת שד של סוג הקו  כאשר רדיוס אפסי או ריק יהיה קו ישר, אחרת קשת
    <h1 style="text-align:right; font-size:100%">
לקוים ישרים בעלי רדיוס אפסי, ערך הרדיוס יוחלף לריק

In [ ]:
straight = 1
curved   = 2

arc['LineType'] = np.where((arc['RADIUS'] == 0) | (arc['RADIUS'].isna()), straight, curved)

arc['RADIUS'] = np.where((arc['LineType'] == straight) & (arc['RADIUS'] == 0.0), np.nan, arc['RADIUS'])
arc['RADIUS'] = arc['RADIUS'].astype('float64')

In [ ]:
print('   Data conversion')

<h1 style="text-align:right; font-size:125%">
:מיזוג נתונים מול נקודות גבול

<h1 style="text-align:right; font-size:100%">
מיזוג ראשון לקבלת מזהה של נקודת התחלה
<h1 style="text-align:right; font-size:100%">
מיזוג שני לקבלת מזהה של נקודת סוף
<h1 style="text-align:right; font-size:100%">
הסרת חזיתות כפולות שנוצאו במיזוג

In [ ]:
BorderPoints = BorderPoints[BorderPoints['PARCEL_SDE'].notna()]
BorderPoints['PARCEL_SDE'] = BorderPoints['PARCEL_SDE'].astype('Int64')
arc[['FNODE_', 'TNODE_']] = arc[['FNODE_', 'TNODE_']].astype('Int64')


arc = arc.merge(BorderPoints, left_on = 'FNODE_', right_on = 'PARCEL_SDE', how = 'left')\
         .rename(columns = {'GlobalID' : 'StartPointUniqueID'})\
         .drop(columns =['PARCEL_SDE', 'FNODE_'])

arc = arc.merge(BorderPoints, left_on = 'TNODE_', right_on = 'PARCEL_SDE', how = 'left')\
         .rename(columns = {'GlobalID' : 'EndPointUniqueID'})\
         .drop(columns = ['PARCEL_SDE', 'TNODE_'])

del BorderPoints

arc = arc.drop_duplicates(subset = ['geometry'], keep = 'first')

<h1 style="text-align:right; font-size:125%">
:מיזוג נתונים מול חלקות ונתוני תהליכי יוצרים

<h1 style="text-align:right; font-size:100%">
מיזוג ראשון לפי חלקה משמאל לחזית
<h1 style="text-align:right; font-size:100%">
מיזוג שני לפי חלקה מימין לחזית
<h1 style="text-align:right; font-size:100%">
כיוון שיש חזיתות שנמצאו להן יותר מתהליך אחד - השדה לתהליך היוצר יתאכלס על ידי התהליך עם התאריך המאוחר יותר. חזיתות ללא מזהה תהליך כלל יקבלו תהליך יוצר פיקטיבי

In [ ]:
arc = arc.merge(Parcels, left_on = 'LPOLY_', right_on = 'PARCEL_SDE_', how = 'left')\
         .drop(columns = ['LPOLY_'])\
         .merge(Parcels, left_on = 'RPOLY_', right_on = 'PARCEL_SDE_', how = 'left', suffixes = ('_L', '_R'))\
         .drop(columns = ['RPOLY_', 'PARCEL_SDE__L', 'PARCEL_SDE__R'])


arc['DataSource'] = arc[['DataSource_L', 'DataSource_R']].max(axis = 1)

arc['CreatedByRecord'] = np.where(arc['DateStatus_L'] > arc['DateStatus_R'], arc['CreatedByRecord_L'], arc['CreatedByRecord_R'])
arc['CreatedByRecord'] = arc['CreatedByRecord'].fillna(CNFG.FabricatedCreateProcessGUID)

arc = arc.drop(columns = ['DataSource_L', 'DataSource_R', 'DateStatus_L', 'DateStatus_R', 'CreatedByRecord_L', 'CreatedByRecord_R'])


del Parcels

<h1 style="text-align:right; font-size:100%">
הגדרת שדה לאורך רשום
<h1 style="text-align:right; font-size:100%">
SRV-ניקוי ערכי אורך רשום של חזיתות שנוצרו מ

In [ ]:
clear_distance_condition = (arc['DataSource'] == 4) & (arc['ProcessYear'] < 2015)
arc['LENGTH'] = np.where(clear_distance_condition, None, arc['LENGTH'])

arc = arc.drop(columns = ['DataSource', 'ProcessYear'])\
         .rename(columns = {'LENGTH' : 'Distance'})

del clear_distance_condition

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("StartPointUniqueID", str), 
                        ("EndPointUniqueID", str),
                        ("Distance", float),
                        ("LineType", int),
                        ("Radius", float),
                        ("CreatedByRecord", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

Fronts = gpd.GeoDataFrame(np.empty(0, dtype = new_columns), geometry = 'geometry', crs = 2039)

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
Fronts['StartPointUniqueID'] = arc['StartPointUniqueID']

Fronts['EndPointUniqueID'] = arc['EndPointUniqueID']

Fronts['Distance'] = arc['Distance'].astype(np.float64)

Fronts['LineType'] = arc['LineType'].astype('Int32')

Fronts['Radius'] = arc['RADIUS'].astype(np.float64)

Fronts['CreatedByRecord'] = arc['CreatedByRecord']

Fronts['geometry'] = arc['geometry']

In [ ]:
del [new_columns, arc]

<h1 style="text-align:right; font-size:100%">
איחוד חזיתות רצף עם חזיתות שומא

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Exporting results')

In [ ]:
Fronts.to_file(CNFG.Results, layer = 'Parcels2DFronts', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')

In [ ]:
print('   Appending results to Parcels2DFronts ')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Fronts-Save as Feature Class.bat']);